# Importing destinations data into Firestore

### Initialize

In [ ]:
from google.cloud import firestore

import pandas as pd
import json

In [ ]:
db = firestore.Client.from_service_account_json("../credentials/stairway-firestore-key.json")

## Batch write

We will have to use a [batch writes](https://cloud.google.com/firestore/docs/manage-data/transactions) for importing larger data sets. Each transaction or batch of writes can write to a maximum of 500 documents. 

NOTE: the [free tier](https://firebase.google.com/docs/firestore/quotas) allows 20.000 writes/deletes per day and up to 1GB of data. So take care to stay below that! Number of free reads per day is 50.000.

In [ ]:
dest = pd.read_csv("../data/destinations.csv").sample(30)
dest.shape

In [ ]:
# transform to JSON
dest_json = json.loads(dest.to_json(orient='index'))

Below will create the batch and commit it. Note, if uploading more than 500 in one batch, the dataframe has to be split. Check out this [blog](https://medium.com/google-cloud/firebase-migrating-data-to-cloud-firestore-using-admin-sdk-6a5184f503c2) of a guy who has already done a batch migrate of over 500 items to Firestore to steal his code.

In [ ]:
# Set the collection
dest_collection = db.collection('destinations-old')

# Define the batch write
batch = db.batch()
for (dest_id, dest_values) in dest_json.items():
    doc = dest_collection.document(dest_id)
    batch.set(doc, dest_values)

# Upload it
batch.commit()

Done.